# OptiGuide Example




Here we give a simple example, as designed and illustrated in the [OptiGuide paper](https://arxiv.org/abs/2307.03875).
While the original paper is designed specifically for supply chain optimization, the general framework can be easily adapted to other applications with coding capacity.




## OptiGuide for Supply Chain Optimization: System Design Overview

The original system design for OptiGuide, tailored for supply chain optimization, is presented below.

The collaboration among three agents -- Coder, Safeguard, and Interpreter -- lies at the core of this system. They leverage a set of external tools and a large language model (LLM) to address users' questions related to supply chain applications. For a comprehensive understanding of the design and data flow, detailed information can be found in the original [paper](https://arxiv.org/abs/2307.03875).


![optiguide system](https://www.beibinli.com/docs/optiguide/optiguide_system.png)


## New Implementation



![](new_design.png)

Advantages of this multi-agent design with autogen:
- Collaborative Problem Solving: The collaboration among the user proxy agent and the assistant agents fosters a cooperative problem-solving environment. The agents can share information and knowledge, allowing them to complement each other's abilities and collectively arrive at better solutions. On the other hand, the Safeguard acts as a virtual adversarial checker, which can perform another safety check pass on the generated code.

- Modularity: The division of tasks into separate agents promotes modularity in the system. Each agent can be developed, tested, and maintained independently, simplifying the overall development process and facilitating code management.

- Memory Management: The OptiGuide agent's role in maintaining memory related to user interactions is crucial. The memory retention allows the agents to have context about a user's prior questions, making the decision-making process more informed and context-aware.



In [2]:
# Install Required Packages
%pip install optiguide

^C
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# test Gurobi installation
import gurobipy as gp
from gurobipy import GRB
from eventlet.timeout import Timeout

# import auxillary packages
import re
import requests  # for loading the example source code
import openai
import os

# import flaml and autogen
from flaml import autogen
from flaml.autogen.agentchat import Agent, UserProxyAgent
from optiguide.optiguide import OptiGuideAgent

from dotenv import load_dotenv
import os
# Path to
load_dotenv('.env')


False

In [ ]:
autogen.oai.ChatCompletion.start_logging()

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": {
            "gpt-4",
            "gpt4",
            "gpt-4-32k",
            "gpt-4-32k-0314",
            "gpt-3.5-turbo",
            "gpt-3.5-turbo-16k",
            "gpt-3.5-turbo-0301",
            "chatgpt-35-turbo-0301",
            "gpt-35-turbo-v0301",
        }
    }
)


Now, let's import the source code (loading from URL) and also some training examples (defined as string blow).

In [ ]:
# Get the source code of our coffee example
# code_url = "https://raw.githubusercontent.com/microsoft/OptiGuide/main/benchmark/application/coffee.py"
code_url = "https://raw.githubusercontent.com/vineesh12344/Codesprint/main/OptiGuide/benchmark/application/port.py"
response  = requests.get(code_url)
# Check if the request was successful
if response.status_code == 200:
    # Get the text content from the response
    code = response.text
else:
    raise RuntimeError("Failed to retrieve the file.")
# code = open(code_url, "r").read() # for local files


# show the first head and tail of the source code
print("\n".join(code.split("\n")[:10]))
print(".\n" * 3)
print("\n".join(code.split("\n")[-10:]))

import time
from gurobipy import GRB, Model

# Example data

capacity_in_supplier = {'supplier1': 150, 'supplier2': 50, 'supplier3': 100}

shipping_cost_from_supplier_to_roastery = {
    ('supplier1', 'roastery1'): 5,
    ('supplier1', 'roastery2'): 4,
.
.
.

# Solve
m.update()
model.optimize()

print(time.ctime())
if m.status == GRB.OPTIMAL:
    print(f'Optimal cost: {m.objVal}')
else:
    print("Not solved to optimality. Optimization status:", m.status)



In [ ]:
# # In-context learning examples.
# example_qa = """
# ----------
# Question: Why is it not recommended to use just one supplier for roastery 2?
# Answer Code:
# ```python
# z = m.addVars(suppliers, vtype=GRB.BINARY, name="z")
# m.addConstr(sum(z[s] for s in suppliers) <= 1, "_")
# for s in suppliers:
#     m.addConstr(x[s,'roastery2'] <= capacity_in_supplier[s] * z[s], "_")
# ```

# ----------
# Question: What if there's a 13% jump in the demand for light coffee at cafe1?
# Answer Code:
# ```python
# light_coffee_needed_for_cafe["cafe1"] = light_coffee_needed_for_cafe["cafe1"] * (1 + 13/100)
# ```

# """

In [ ]:
# In-context learning examples.
example_qa = """
----------
Question: Why is it not recommended to only use one warehouse for berth 1?
Answer Code:
```python
z = m.addVars(warehouses, vtype=GRB.BINARY, name="z")
m.addConstr(sum(z[w] for w in warehouses) <= 1, "_")
for w in warehouses:
    m.addConstr(shipping_costs['Berth1', w] <= warehouseCapacities[w] * z[w], "_")
```

----------
Question: What if there's a 50% jump in the capacity of warehouse 1?
Answer Code:
```python
warehouses["Warehouse1"] = warehouses["Warehouse1"] * (1 + 50/100)
```

"""

Now, let's create an OptiGuide agent and also a user.

For the OptiGuide agent, we only allow "debug_times" to be 1, which means it can debug its answer once if it encountered errors.

In [ ]:
%%capture
agent = OptiGuideAgent(name="OptiGuide Coffee Example",
                  source_code=code,
                   debug_times=1,
                  example_qa="",
                llm_config={
        "request_timeout": 600,
        "seed": 42,
        "config_list": config_list,
    })

user = UserProxyAgent("user", max_consecutive_auto_reply=0,
                         human_input_mode="NEVER", code_execution_config=False)


Now, let's create a user's question.

In [ ]:
message = user.initiate_chat(agent, message="What if we prohibit shipping from supplier 1 to roastery 2?")


user (to OptiGuide Coffee Example):

What if we prohibit shipping from supplier 1 to roastery 2?

--------------------------------------------------------------------------------
OptiGuide Coffee Example (to writer):


Answer Code:


--------------------------------------------------------------------------------


AuthenticationError: No API key provided. You can set your API key in code using 'openai.api_key = <API-KEY>', or you can set the environment variable OPENAI_API_KEY=<API-KEY>). If your API key is stored in a file, you can point the openai module at it with 'openai.api_key_path = <PATH>'. You can generate API keys in the OpenAI web interface. See https://platform.openai.com/account/api-keys for details.

In [ ]:
user.initiate_chat(agent, message="What is the impact of supplier1 being able to supply only half the quantity at present?")


user (to OptiGuide Coffee Example):

What is the impact of supplier1 being able to supply only half the quantity at present?

--------------------------------------------------------------------------------
OptiGuide Coffee Example (to writer):


Answer Code:


--------------------------------------------------------------------------------
writer (to OptiGuide Coffee Example):

```python
# Update the capacity of supplier1 to half the original value
new_capacity = capacity_in_supplier['supplier1'] / 2
capacity_in_supplier.update({'supplier1': new_capacity})

# Update the supply constraint for supplier1
model.remove(model.getConstrByName('supply_supplier1'))
model.addConstr(
    sum(x[i] for i in shipping_cost_from_supplier_to_roastery.keys()
        if i[0] == 'supplier1') <= capacity_in_supplier['supplier1'], "supply_supplier1")

# Solve the updated model
model.optimize()

# Print the new optimal cost
if model.status == GRB.OPTIMAL:
    updated_cost = model.objVal
else:
    updated_co